In [1]:
from codex import Codex

In [2]:
filename = 'pw.in'
database_dir = f'database/'
version = '7.2'
# Make .dft-tutor directory in current dir if it doesn't exist
Codex(filename, database_dir, version)
